In [2]:
"""
Tarang TO DO

1. FIX GITIGNORE
    Operates locally.
""";

In [24]:
# Importing Webscraping, data managment and visualization libraries.

import pandas as pd, matplotlib.pyplot as plt, seaborn as sns

# Web–scraping & cleaning libs.
import requests
from bs4 import BeautifulSoup

# NLP Libs.
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,\
AdaBoostClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,\
ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
#
#import xgboost as xgb

[nltk_data] Downloading package punkt to /Users/tarang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tarang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
""" This is how we will start to extract the lyrics from the song webpage.
"""

url = 'https://www.lyrics.com/lyric/31024783/The+Strokes/Heart+In+a+Cage'
req = requests.get(url)

soup = BeautifulSoup(req.text)
lyrics = soup.find("pre",{'id' : 'lyric-body-text'}).text
lyrics

"Well, I don't feel better when I'm fucking around\nAnd I don't write better when I'm stuck in the ground\nSo don't teach me a lesson 'cause I've already learned\nYeah, the sun will be shining and my children will burn\n\nOh, the heart beats in its cage\n\nI don't want what you want, I don't feel what you feel\nSee, I'm stuck in a city but I belong in a field\nYeah, we got left, left, left, left, left, left, left\nNow it's three in the morning and you're eating alone\n\nOh, the heart beats in its cage\n\nAll our friends, they're laughing at us\nAll of those you loved, you mistrust\nHelp me, I'm just not quite myself\nLook around, there's no one else left\n\nI went to the concert and I fought through the crowd\nGuess I got too excited when I thought you were around\nOh, he gets left, left, left, left, left, left, left\nAnd I'm sorry you were thinking I would steal your fire\n\nOh, the heart beats in its cage\nYes, the heart beats in its cage\n\nAnd the heart beats in its cage"

In [7]:
# \n is a line \n\n is a stanza.
#<div class="clearfix"><h3 class="artist-album-label">

In [10]:
"""
SHOULD I USE SUB FUNCTIONS
""";
user_id = open('./data/lyrics_user.txt', mode = 'r').read()
token = open('./data/lyrics_key.txt', mode = 'r').read()

artist = input('Enter artist here : ')
# if not redirected to artist page (instead results of ARTISTS, return message to include articles in name)
url = 'https://www.lyrics.com/artist/'+str(artist.replace(' ', '+'))
req = requests.get(url)
soup = BeautifulSoup(req.text)
tables = soup.find_all('table', {'class': 'tdata'})
songs = []
for table in tables:
    for row in table.find('tbody').find_all('tr'):
        songs.append(row.find('a').text)
songs = pd.DataFrame(set(songs), columns = ['Titles'])
print(songs)
# how to display FULL disc on Web Pge w/0 elipsis?
album = input('Choose a title by index number : ')
song = str(songs.loc[int(album),'Titles'])
#
# Getting album url to pull tracklist and search lyrics for each.
#
url = 'https://www.stands4.com/services/v2/lyrics.php'
params = {'uid' : str(user_id),
         'tokenid' : str(token),
         'term' : str(song),
         'format': 'json'}
req = requests.get(url, params)
req.json()
#
# Log album name and use url from dict to nav to page and pull track list for further querying
#

Enter artist here : the strokes
                                        Titles
0                              Ode to the Mets
1    New York City Cops [Live in Iceland 2002]
2                        You Talk Way Too Much
3                                 Happy Ending
4                                       Hawaii
..                                         ...
100                    Meet Me In the Bathroom
101                       Not the Same Anymore
102                    Meet Me in the Bathroom
103                            Hard to Explain
104                                   Juicebox

[105 rows x 1 columns]
Choose a title by index number : 10


{'result': [{'song': 'Ask Me Anything',
   'song-link': 'https://www.lyrics.com/lyric/8505390/The+Strokes/Ask+Me+Anything',
   'artist': 'The Strokes',
   'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260',
   'album': 'First Impressions of Earth',
   'album-link': 'https://www.lyrics.com/album/812605/First Impressions of Earth'},
  {'song': 'Ask Me Anything',
   'song-link': 'https://www.lyrics.com/lyric/28316031/The+Strokes/Ask+Me+Anything',
   'artist': 'The Strokes',
   'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260',
   'album': 'First Impressions of Earth [LP]',
   'album-link': 'https://www.lyrics.com/album/815144/First Impressions of Earth [LP]'},
  {'song': 'Ask Me Anything',
   'song-link': 'https://www.lyrics.com/lyric/31024779/The+Strokes/Ask+Me+Anything',
   'artist': 'The Strokes',
   'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260',
   'album': 'First Impressions of Earth',
   'album-link': 'https://www.lyrics.com/album/8

In [11]:
json_results = req.json()

## Code to capture album data for songs with more than 7 titles

In [12]:
##basic loop that goes through the json output/query result.
counter = 0
for counter in range(len(json_results["result"])):
    ##if json_code output == to artist name input: then grab the album name
    if json_results["result"][counter]['artist'].lower() == artist.lower():
        
        ##code that adds all the songs within the album
        url = json_results["result"][counter]['album-link']
        req = requests.get(url)
        soup = BeautifulSoup(req.text)
        tables = soup.find_all('table', {'class': 'tdata'})
        album_songs = []
        for table in tables:
            for row in table.find('tbody').find_all('tr'):
                
                album_songs.append(row.find('a').text)
        if len(album_songs) >= 7:
            
            ##CODE FOR STORING THE ALBUM AND ITS SONGS INTO A DATA FRAME THAT CONTAINS ALBUM, SONG NAME, SONG URL, 
            ##ARTIST GOES HERE
            
            ##This code grabs all the song urls within the album            
            for table in tables:
                link_list = [a['href'] for a in soup.find_all('a', href=True)]
                lyrics_link = []
                for link in link_list:
                    if link.startswith('/lyric/'):
                        lyrics_link.append(link)
                        
            full_link_list = []
            for link in lyrics_link:
                full_link_list.append('https://www.lyrics.com/'+link)
                
            ##Thies creates the dataframe of the 
            album_data = pd.DataFrame({'Artist':json_results["result"][counter]['artist'],
                           'Album':json_results["result"][counter]['album'],
                           'Song Link':full_link_list,
                           'Song Name':album_songs})            
            
                
 
        counter += 1

In [13]:
album_data

,Artist,Album,Song Link,Song Name
0,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664477/Is+This+It,Is This It
1,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664476/The+Mod...,The Modern Age
2,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664475/Soma,Soma
3,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664474/Barely+...,Barely Legal
4,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664473/Someday,Someday
5,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664472/Alone%2...,"Alone, Together"
6,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664471/Last+Nite,Last Nite
7,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664470/Hard+to...,Hard to Explain
8,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664469/When+It...,When It Started
9,The Strokes,The Complete Collection,https://www.lyrics.com//lyric/32664468/Trying+...,Trying Your Luck


In [43]:
artist

'the strokes'

In [47]:
"""
INITIAL ARTIST PAGE QUERY
"""


res = req.json()
df = res['result']
df
new_df = {}
for i in df:
    for key, value in i.items():
        if value.lower() == artist.lower():
            new_df.update(i)
            break
print(new_df)

"""
How to deal w/ duplicative results on an EP?
    Attribution for albums 'studio', etc.?
""";

{'song': 'Barely Legal', 'song-link': 'https://www.lyrics.com/lyric/35821913/The+Strokes/Barely+Legal', 'artist': 'The Strokes', 'artist-link': 'https://www.lyrics.com/artist/The+Strokes/478260', 'album': 'Is This It [180 Gram Vinyl]', 'album-link': 'https://www.lyrics.com/album/2367673/Is This It [180 Gram Vinyl]'}


In [ ]:
"""
Return tracklist from album page
go to track page & extract lyrics for each
dev DF & attribution
model
"""

In [46]:
album = input('Choose a title by index number : ')
str(songs.loc[int(album),'Titles'])

Choose a title by index number : 6


'New York City Cops'

In [41]:
# only works for scalars
#songs.at(5, 'Titles')

TypeError: '_AtIndexer' object is not callable

In [26]:
token = pd.read_csv('./data/lyrics_key.txt', )
print(token)

Empty DataFrame
Columns: [5fGYJwlI810Y5kTs]
Index: []


In [27]:
""" Use this to start to document attributes of music related to song req.
We can store information about the album, artist and song name while navigating to the album page.
From album page we can then loop through song tags and request to their respective pages. Lyric Query is above.
"""

# FN passing through index # of desired song. will work off assumption that fav song will be on album w/ sim tracks

user_id = pd.read_csv('./data/lyrics_user.txt')
token = pd.read_csv('./data/lyrics_key.txt')
song = 'Selfless'

url = 'https://www.stands4.com/services/v2/lyrics.php'
params = {'uid' : str(user_id),
         'tokenid' : str(token),
         'term' : str(song),
         'format': 'json'}
req = requests.get(url, params)
req.json()

#### Should be sub fn using specified artist to refine results to single dict. w/ key:val pairs to document & nav.

{'error': 'Invalid Token'}

save info to csv for modeling

---

### MW API

---

In [15]:
thesarus_key = open('./data/mw_key.txt', mode = 'r').read()
thesarus_key

'6d207136-9763-48c2-b218-6369e35e2036'

In [16]:
# ADD LINK TO DICTIONARY API DOCS
mw_url = 'https://www.dictionaryapi.com/api/v3/references/thesaurus/json/'#umpire?key=6d207136-9763-48c2-b218-6369e35e2036'
params = {
    'key' : thesarus_key
         }
# loop through list of nouns, adj, etc.

example = ['umpire', 'pussy']

def get_meta(words):
    params = {
    'key' : thesarus_key
         }
    for word in words:
        mw_url = 'https://www.dictionaryapi.com/api/v3/references/thesaurus/json/'
        mw_url += word
        word = requests.get(mw_url, params)
        print(word.url)



In [18]:
get_meta(example)

https://www.dictionaryapi.com/api/v3/references/thesaurus/json/umpire?key=6d207136-9763-48c2-b218-6369e35e2036
https://www.dictionaryapi.com/api/v3/references/thesaurus/json/pussy?key=6d207136-9763-48c2-b218-6369e35e2036


In [19]:
req = requests.get(mw_url, params)

In [20]:
print(req.url)

https://www.dictionaryapi.com/api/v3/references/thesaurus/json/?key=6d207136-9763-48c2-b218-6369e35e2036


In [21]:
req.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)